In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Store csv created in part one into a DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")
city_data_df.head()

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,hobart,75,AU,1569398154,58,-42.88,147.33,55.00,8.05
1,1,ahipara,27,NZ,1569398227,100,-35.17,173.16,53.01,37.36
2,2,thompson,90,CA,1569398227,87,55.74,-97.86,44.60,10.29
3,3,hermanus,0,ZA,1569398227,54,-34.42,19.24,71.01,14.00
4,4,yellowknife,75,CA,1569398227,100,62.45,-114.38,35.60,6.93


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key)

In [4]:
# Heatmap of humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=5)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# Narrow down cities that fit criteria and drop any results with null values
narrowed_city_df = city_data_df.loc[(city_data_df["Max Temp"] < 80) & (city_data_df["Max Temp"] > 70) \
                                    & (city_data_df["Wind Speed"] < 10) \
                                    & (city_data_df["Cloudiness"] == 0)].dropna()
narrowed_city_df

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
9,9,cape town,0,ZA,1569397980,12,-33.93,18.42,72.00,3.18
69,69,atocha,0,AR,1569398236,51,-24.82,-65.48,71.60,3.36
89,89,qinhuangdao,0,CN,1569398239,57,39.94,119.59,74.03,6.35
263,263,humaita,0,AR,1569398262,51,-24.93,-65.37,71.60,3.36
277,277,kuhdasht,0,IR,1569398264,13,33.53,47.61,78.80,4.70
317,317,la ciotat,0,FR,1569398269,77,43.17,5.60,79.00,8.05
321,321,trapani,0,IT,1569398237,61,38.02,12.54,78.01,8.05
377,377,manbij,0,SY,1569398277,36,36.53,37.96,77.00,4.70
387,387,penne,0,IT,1569398279,82,42.45,13.93,73.00,1.12
393,393,cayenne,0,GF,1569398279,100,4.94,-52.33,73.40,2.33


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
# Create DataFrame called hotel_df to store hotel names along with city, country and coordinates
hotel_df = narrowed_city_df[["City", "Country", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Lat,Lng,Hotel Name
9,cape town,ZA,-33.93,18.42,
69,atocha,AR,-24.82,-65.48,
89,qinhuangdao,CN,39.94,119.59,
263,humaita,AR,-24.93,-65.37,
277,kuhdasht,IR,33.53,47.61,
317,la ciotat,FR,43.17,5.60,
321,trapani,IT,38.02,12.54,
377,manbij,SY,36.53,37.96,
387,penne,IT,42.45,13.93,
393,cayenne,GF,4.94,-52.33,


In [7]:
# Set parameters to search for a hotel
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}

# Iterate through 
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    # Use the search term: "Hotel" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
    # convert to json
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

hotel_df

,City,Country,Lat,Lng,Hotel Name
9,cape town,ZA,-33.93,18.42,Southern Sun Waterfront Cape Town
69,atocha,AR,-24.82,-65.48,Hotel Aybal
89,qinhuangdao,CN,39.94,119.59,Qinhuangdao Peninsula Seasons Hotel And Apartment
263,humaita,AR,-24.93,-65.37,Paraiso
277,kuhdasht,IR,33.53,47.61,KamranM
317,la ciotat,FR,43.17,5.60,Hotel ibis La Ciotat
321,trapani,IT,38.02,12.54,Hotel Vittoria
377,manbij,SY,36.53,37.96,Manbej City Hotel
387,penne,IT,42.45,13.93,Castello Chiola Hotel
393,cayenne,GF,4.94,-52.33,Hôtel Le Dronmi


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [9]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))